In [1]:
import pandas as pd

df_road = pd.read_csv("/Users/2020/road-to-dong/pos00005m.csv")
df_dong = pd.read_csv("/Users/2020/road-to-dong/pos00006m.csv")

df_road = df_road[['rd_nm', 'bld_hmno', 'bld_hsno', 'lgdng_cd']]
df_dong = df_dong[df_dong['ctpv_nm']=='서울특별시']

def bld_split(bd_nm):
    if '-' in bd_nm:
        bld_hmno, bld_hsno = bd_nm.split('-')
    else:
        bld_hmno = bd_nm
        bld_hsno = '0'
    return bld_hmno, bld_hsno

# raod : str type
def get_rd_bd(road_addr):
    temp_list = road_addr.split()
    rd_nm = temp_list[2]
    bld_hmno, bld_hsno = bld_split(temp_list[3])
    if bld_hmno.isdecimal() & bld_hsno.isdecimal():
        return rd_nm, int(bld_hmno), int(bld_hsno)
    else:
        return rd_nm, 0, 0  

# road_bd 예시 : '테헤란로6길', 30, 0 ( '길' '건물본번' '건물부번' type:str, int, int)
def rd_bd_to_dong(rd_nm, bld_hmno, bld_hsno):
    if bld_hmno==0:
        return '검색불가' # 오류 : 잘못 기록된 도로명
    df_temp = df_road[df_road['rd_nm']==rd_nm]
    df_temp = df_temp[df_temp['bld_hmno']==bld_hmno]
    df_temp = df_temp[df_temp['bld_hsno']==bld_hsno]
    if len(df_temp) == 0:
        return '검색불가' # 검색 테이블에 존재x
    for i in range(len(df_temp['lgdng_cd'])):
        lgdng_cd = df_temp['lgdng_cd'].iat[i]
        df_temp2 = df_dong[df_dong['lgdng_cd']==lgdng_cd]
        if len(df_temp2) != 0:
            return df_temp2.iloc[0]['lgdng_nm']
    return '검색불가'# 서울 외 지역

# 도로명주소(상세)-> 법정동 이름
def road_address_to_dong(road_addr):
    return rd_bd_to_dong(*get_rd_bd(road_addr))

# road_address to dong (series)
# .reset_index(drop=True)를 통해서 index를 0부터 시작하게 셋팅
def rtds(pd_series):
    for i in range(len(pd_series)):
        pd_series[i] = road_address_to_dong(pd_series[i])
    return pd_series

### Test

In [2]:
road_address_to_dong('서울특별시 구로구 구일로 154-17 101동 101호')

'구로동'

In [3]:
pd.set_option('display.max_row', 1000)

In [4]:
test_series = pd.read_csv("/Users/2020/road-to-dong/서울특별시 어린이집 정보(표준 데이터).csv", encoding='cp949')
test_series = test_series['상세주소'].copy()
test_series = test_series[:500]
test_series

0                                   서울특별시 서초구 사평대로 310-4
1                                      서울특별시 서초구 효령로 391
2                            서울특별시 서초구 방배로 21 방배그랑자이어린이집
3                                  서울특별시 송파구 마천로28길 3 1층
4                                    서울특별시 구로구 부광로 88 1층
5                       서울특별시 광진구 능동로 69 어린이집(자양동자양호반써밋)
6           서울특별시 마포구 마포대로11길 50 409동 1층(공덕동 공덕래미안4차아파트)
7                                    서울특별시 송파구 오금로 62 1층
8                                    서울특별시 구로구 고척로33길 43
9                   서울특별시 강남구 테헤란로98길 12 3층(대치동현대백화점 본사)
10                                   서울특별시 강남구 논현로99길 21
11                           서울특별시 금천구 벚꽃로 100 105동 102호
12                            서울특별시 중랑구 용마산로136길 160 관리동
13                          서울특별시 서초구 사평대로 240 502동 107호
14                  서울특별시 동작구 동작대로39길 39 이수푸르지오더프레티움 관리동
15                                   서울특별시 서초구 서초중앙로 206
16                                    서울특별시 송파구 위례송파로 89
17                서울특별시 금천구 벚꽃로

In [ ]:
# 분당 200개 속도 (코랩 무료버전 정도 성능)
test_series = rtds(test_series)

In [ ]:
test_series

In [ ]:
# 총 500개
test_series.value_counts()

### Test2

In [5]:
pd.set_option('display.max_row', 1000)

In [6]:
test_series2 = pd.read_csv("/Users/2020/road-to-dong/subway_road_test.csv", encoding='cp949')
test_series2 = test_series2['도로명주소'].copy()


In [7]:
test_series2

0                   경기도 동두천시 평화로 2925
1                  경기도 동두천시 평화로 2687 
2                   경기도 동두천시 평화로 2539
3                   경기도 동두천시 동두천로 228
4                   경기도 동두천시 평화로 2285
5                    경기도 양주시 화합로 1356
6               경기도 양주시 고덕로 139번길 317
7                     경기도 양주시 평화로 919
8                    경기도 의정부시 평화로 757
9                    경기도 의정부시 평화로 633
10                   경기도 의정부시 평화로 525
11                   경기도 의정부시 평화로 363
12                   경기도 의정부시 평화로 221
13               서울특별시 도봉구 도봉로 964-33
14               서울특별시 도봉구 도봉로 170길 2
15              서울특별시 도봉구 도봉로 150다길 3
16                서울특별시 도봉구 마들로11길 77
17                  서울특별시 도붕구 덕릉로 376
18                서울특별시 노원구 월계로53길 40
19                 서울특별시 노원구 석계로 98-2
20                  서울특별시 노원구 화랑로 341
21                 서울특별시 동대문구 한천로 472
22                  서울특별시 동대문구 휘경로 27
23                 서울특별시 동대문구 회기로 196
24               서울특별시 용산구 한강대로77길 25
25               서울특별시 용산구 한강대로23길 55
26          

In [8]:
test_series2 = test_series2[test_series2.str.split().str[0]=='서울특별시']

In [9]:
test_series2

13          서울특별시 도봉구 도봉로 964-33
14          서울특별시 도봉구 도봉로 170길 2
15         서울특별시 도봉구 도봉로 150다길 3
16           서울특별시 도봉구 마들로11길 77
17             서울특별시 도붕구 덕릉로 376
18           서울특별시 노원구 월계로53길 40
19            서울특별시 노원구 석계로 98-2
20             서울특별시 노원구 화랑로 341
21            서울특별시 동대문구 한천로 472
22             서울특별시 동대문구 휘경로 27
23            서울특별시 동대문구 회기로 196
24          서울특별시 용산구 한강대로77길 25
25          서울특별시 용산구 한강대로23길 55
26            서울특별시 동작구 노량진로 151
27          서울특별시 영등포구 여의대방로 300
28           서울특별시 영등포구 영등포로 327
29            서울특별시 영등포구 경인로 846
30        서울특별시 구로구 새마로 지하117-21
31           서울특별시 구로구 구로중앙로 174
32             서울특별시 구로구 구일로 133
33           서울특별시 구로구 경인로40길 47
34          서울특별시 구로구 경인로 20길 13
35             서울특별시 구로구 부일로 872
52             서울특별시 금천구 벚꽃로 309
53             서울특별시 금천구 벚꽃로 115
54         서울특별시 금천구 시흥대로 63길 91
131        서울특별시 용산구 한강대로 23길 55
132         서울특별시 용산구 이촌로 71길 42
133           서울특별시 용산구 서빙고로 238
134      서울특별시 용산구 독서당로 6길 12-13
135       

In [17]:
test_series2 = test_series2.reset_index(drop=True)
test_series2

0          서울특별시 도봉구 도봉로 964-33
1          서울특별시 도봉구 도봉로 170길 2
2         서울특별시 도봉구 도봉로 150다길 3
3           서울특별시 도봉구 마들로11길 77
4             서울특별시 도붕구 덕릉로 376
5           서울특별시 노원구 월계로53길 40
6            서울특별시 노원구 석계로 98-2
7             서울특별시 노원구 화랑로 341
8            서울특별시 동대문구 한천로 472
9             서울특별시 동대문구 휘경로 27
10           서울특별시 동대문구 회기로 196
11         서울특별시 용산구 한강대로77길 25
12         서울특별시 용산구 한강대로23길 55
13           서울특별시 동작구 노량진로 151
14         서울특별시 영등포구 여의대방로 300
15          서울특별시 영등포구 영등포로 327
16           서울특별시 영등포구 경인로 846
17       서울특별시 구로구 새마로 지하117-21
18          서울특별시 구로구 구로중앙로 174
19            서울특별시 구로구 구일로 133
20          서울특별시 구로구 경인로40길 47
21         서울특별시 구로구 경인로 20길 13
22            서울특별시 구로구 부일로 872
23            서울특별시 금천구 벚꽃로 309
24            서울특별시 금천구 벚꽃로 115
25        서울특별시 금천구 시흥대로 63길 91
26        서울특별시 용산구 한강대로 23길 55
27         서울특별시 용산구 이촌로 71길 42
28           서울특별시 용산구 서빙고로 238
29      서울특별시 용산구 독서당로 6길 12-13
30             서울특별시 성동구 동호로 21
31      

In [18]:
test_series2 = rtds(test_series2)
test_series2

0      검색불가
1      검색불가
2      검색불가
3        창동
4        창동
5       월계동
6       월계동
7       월계동
8       이문동
9       이문동
10      휘경동
11      갈월동
12    한강로3가
13     노량진동
14      신길동
15      신길동
16     영등포동
17     검색불가
18      구로동
19      구로동
20      개봉동
21     검색불가
22      온수동
23      가산동
24      가산동
25     검색불가
26     검색불가
27     검색불가
28     서빙고동
29     검색불가
30      옥수동
31      응봉동
32      행당동
33      전농동
34      휘경동
35      중화동
36      상봉동
37     검색불가
38      망우동
39     검색불가
40      도화동
41     노고산동
42      동교동
43     남가좌동
44      수색동
45      수색동
46    봉래동2가
47      신촌동
48      월계동
49      전농동
50      휘경동
51      중화동
52      상봉동
53     검색불가
54      망우동
55      전농동
56      행당동
57    성수동1가
58     압구정동
59      삼성동
60      삼성동
61      대치동
62      대치동
63      도곡동
64      개포동
65      개포동
66      일원동
67      수서동
68      장지동
Name: 도로명주소, dtype: object